In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
df = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv').sample(frac=1, random_state=1)
train,test_data = train_test_split(df,test_size=0.2, random_state=1)
print(train['target'].value_counts())

In [ ]:
train_data = train.head(len(train)//2)
# train_data = pd.concat([train.loc[train['target']==1].tail(60000), train.loc[train['target']==0].tail(60000)], axis=0)

In [ ]:
model_name = "distilroberta-base"
max_length = 128

In [ ]:
class_weights = compute_class_weight('balanced', np.unique(train_data['target']), train_data['target'])
print("Class Weights:",class_weights)
weights = torch.tensor(class_weights,dtype=torch.float)
weights = weights.to("cuda")
cross_entropy  = nn.NLLLoss(weight=weights)

In [ ]:
# from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
# from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification

# load the tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_data['question_text'].apply(str).tolist(),
                                                                    train_data['target'].apply(int).tolist(), train_size=0.8)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# convert our tokenized data into a torch Dataset
train_dataset = CustomDataset(train_encodings, train_labels)
valid_dataset = CustomDataset(val_encodings, val_labels)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")
# model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")
# model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
from transformers import Trainer, TrainingArguments

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # print('labels shape', labels.shape)
        # print('logits shape', logits.shape)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(class_weights,device="cuda").float())
        loss = loss_fct(logits,labels.long())
        return (loss, outputs) if return_outputs else loss

# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=1,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size per device during training
#     per_device_eval_batch_size=20,   # batch size for evaluation
#     warmup_steps=1000,                 # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
#     # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
#     logging_steps=1000,               # log & save weights each logging_steps - used to be 50
#     evaluation_strategy="steps",     # evaluate each `logging_steps`
# )

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=50,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=False,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=1000,               # log & save weights each logging_steps
    save_steps=1000,
    save_total_limit=5,
    fp16=True,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer = WeightedTrainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
# train the model
trainer.train()

In [ ]:
# evaluate the current model after training
trainer.evaluate()

In [ ]:
def get_prediction_proba(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs

def get_prediction(text):
    return get_prediction_proba(text).argmax().item()

In [ ]:
test = test_data.sample(4000)
test_data['target'].value_counts()
# test = pd.concat([test_data[test_data['target']==1].head(1000), test_data[test_data['target']==0].head(1000)], axis=0)

In [ ]:
test['pred'] = test['question_text'].apply(get_prediction)

In [ ]:
sklearn.metrics.f1_score(test['target'], test['pred'])

In [ ]:
accuracy_score(test['target'], test['pred'])

In [ ]:
test_final = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
test_final['pred'] = test_final['question_text'].apply(get_prediction)

In [ ]:
submission = test_final.drop(columns=['question_text'])
submission.to_csv('submission.csv')